In [7]:
import pandas as pd
data = pd.read_csv(
            "https://bit.ly/3X2qp1r"
)

In [5]:
!pip install pykeen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.0/704.0 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 30.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-m

In [8]:
from pykeen.triples import TriplesFactory
tf = TriplesFactory.from_labeled_triples(data[["source_name", "type", "target_name"]].values,
)

In [9]:
training, testing, validation = tf.split([0.8, 0.1, 0.1], random_state=0)

INFO:pykeen.triples.splitting:done splitting triples to groups of sizes [430312, 56172, 56172]


In [10]:
import gc
del data
gc.collect()

30

In [12]:
from pykeen.pipeline import pipeline
result = pipeline(
    training=training,
    testing=testing,
    validation=validation,
    model="PairRE",
    model_kwargs=dict(embedding_dim=32),
    training_kwargs=dict(batch_size=100, num_epochs=2),
    stopper="early",
    epochs=2,
   random_seed=0,
    device="cuda"
)

INFO:pykeen.pipeline.api:Using device: cuda
INFO:pykeen.stoppers.early_stopping:Inferred checkpoint path for best model weights: /root/.data/pykeen/checkpoints/best-model-weights-48f6d27b-9ce6-4fe8-b355-dea5d6ac929c.pt


Training epochs on cuda:0:   0%|          | 0/2 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/4494 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/4494 [00:00<?, ?batch/s]

Evaluating on cuda:0:   0%|          | 0.00/56.2k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 5.55s seconds


In [39]:
from pykeen.predict import predict_target
pred = predict_target(
    result.model,
    head="Acetylsalicylic acid",
    relation="treats",
    triples_factory=result.training).df.head()

In [40]:
pred

,tail_id,score,tail_label
19835,19835,-0.976345,coronary artery disease
19920,19920,-1.043491,type 2 diabetes mellitus
19817,19817,-1.135463,asthma
19881,19881,-1.147135,obesity
19912,19912,-1.157378,systemic lupus erythematosus


In [15]:
pred_filtered = pred.filter_triples(result.training).df
print(pred_filtered.head())

       tail_id     score                    tail_label
19920    19920 -1.043491      type 2 diabetes mellitus
19817    19817 -1.135463                        asthma
19881    19881 -1.147135                       obesity
19912    19912 -1.157378  systemic lupus erythematosus
19906    19906 -1.158591                       sarcoma


## Extract a metric of your interest, here we choose hits@k with k=5

In [17]:
result.get_metric('hits@5')

0.02095349996439507

In [24]:
!pip install openai --upgrade
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 24.4 MB/s eta 0:00:00


In [27]:
!pip install langchain-community langchain-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.5 MB/s eta 0:00:00


In [34]:
import os
import openai
file_path = '/content/sample_data/api_key.txt'

# Read the API key from the file
with open(file_path, 'r') as file:
    api_key = file.read().strip()  # Read the API key and remove any leading/trailing whitespace

# Set the environment variable
os.environ['OPENAI_API_KEY'] = api_key

from openai import OpenAI
client = OpenAI()

In [35]:
from sqlalchemy.engine.url import URL
from langchain.llms import OpenAI
from langchain.indexes import GraphIndexCreator

import os
os.environ['OPENAI_API_KEY'] = api_key
openai_api_key = os.getenv("OPENAI_API_KEY")

user_query = "Acetylsalicylic acid, treats, ?."

index_creator = GraphIndexCreator(llm=OpenAI(temperature=0, openai_api_key=openai_api_key))
graph = index_creator.from_text(user_query)
head=graph.get_triples()[0][0]
relation=graph.get_triples()[0][2]

In [44]:
prompt_role = '''As an acedemic researcher working on drug repurposing and indication exapnsion.
Provide evidence of this treament, or relationship from the scientific publications.
You should provide summary of the studies, outcome of study and title of the study. Please provide as many studies as possible. But they should be based on scientific facts'''


prompt = f"According to knowledge graph query {head} is most likely to {relation} following providers/n.{prompt_role}\n {pred.to_string(index=False)}"
response = client.chat.completions.create(model="gpt-3.5-turbo", temperature = 0, messages=[{"role": "user", "content": prompt}])

model_response=response.choices[0].message.content
print(model_response)

Study 1: 
Title: "Aspirin for primary prevention of cardiovascular events in people with diabetes: meta-analysis of randomised controlled trials" 
Summary: This meta-analysis study found that acetylsalicylic acid (aspirin) can be effective in reducing the risk of cardiovascular events in individuals with type 2 diabetes mellitus. 
Outcome: The study concluded that aspirin therapy could be beneficial for primary prevention of cardiovascular events in diabetic patients. 

Study 2: 
Title: "Aspirin for the primary prevention of cardiovascular events: a systematic review and meta-analysis comparing patients with and without diabetes" 
Summary: This systematic review and meta-analysis compared the effectiveness of aspirin for primary prevention of cardiovascular events in patients with and without diabetes. 
Outcome: The study found that aspirin therapy was associated with a reduced risk of cardiovascular events in both diabetic and non-diabetic patients, suggesting its potential benefit in